In [ ]:
import os

os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
def build_chain(chain, llm, parser):
    return(chain | llm | parser)

### Writing test scenarios

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt1 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that writes test scenarios for software requirements in {language}.",
        ),
        (
            "human", 
            "{requirement}"
        ),
    ]
)

In [ ]:
chain1 = build_chain(prompt1, llm, parser)

test_scenario = chain1.invoke(
    {
        "language": "English",
        "requirement": "The application shall match the color of the schema set forth by Department of Homeland Security",
    }
)

### Writing tests

In [ ]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that writes tests in python based on test scenarios.",
        ),
        (
            "human", 
            "Write tests for the following test scenario: {test_scenario}"
        ),
    ]
)

chain2 = build_chain(prompt2, llm, parser)

test = chain2.invoke(
    {
        "test_scenario": test_scenario
    }
)

In [ ]:
test

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain1 = LLMChain(
    llm=llm, 
    prompt=prompt1, 
    output_key="test_scenario"
)


In [ ]:
chain2 = LLMChain(
    llm=llm, 
    prompt=prompt2, 
    output_key="tests"
)


In [ ]:
from langchain.chains import SequentialChain

In [ ]:
overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["language","requirement"],
    output_variables=["test_scenario","tests"],
    verbose=True
)

In [ ]:
requirement="The application shall match the color of the schema set forth by Department of Homeland Security",
overall_chain({'language':'english', 'requirement':requirement})

In [ ]:

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)


In [ ]:
conversation.predict(input="Hi, I'm Allan")